In [1]:
import pandas as pd
import json
import requests
from config import gkey

In [2]:
city_counts = pd.read_csv("Resources/DrugsbyCityWLatLng.csv")
city_counts.head()

,Unnamed: 0,City,Unnamed: 0.1,ID,Date,DateType,Age,Sex,Race,ResidenceCity,...,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine Not Heroin,Hydromorphone,Other,MannerofDeath,Latitude,Longitude
0,1,AMSTON,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,41.625420,-72.343059
1,2,ANDOVER,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,3,41.737321,-72.370360
2,3,ANSONIA,23,23,23,23,23,23,23,23,...,8,5,1,0,1,0,2,23,41.346208,-73.078997
3,4,ASHFORD,6,6,6,6,6,6,6,6,...,2,0,1,0,0,0,0,6,41.873066,-72.121616
4,5,AVON,5,5,5,5,5,5,5,5,...,2,0,0,0,0,0,0,5,41.809620,-72.830515


In [3]:
# Limit to desired data
city_counts = city_counts.iloc[:,[1,22,23,24,25,26,27,28,29,30,31,32,33,35,37,38]]
city_counts

,City,Heroin,Cocaine,Fentanyl,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine Not Heroin,Other,Latitude,Longitude
0,AMSTON,1,0,1,0,0,0,0,0,0,0,0,0,0,41.625420,-72.343059
1,ANDOVER,1,0,2,2,0,1,0,0,0,0,0,0,0,41.737321,-72.370360
2,ANSONIA,6,4,6,8,1,5,0,8,5,1,0,1,2,41.346208,-73.078997
3,ASHFORD,3,4,4,0,0,3,0,2,0,1,0,0,0,41.873066,-72.121616
4,AVON,1,2,3,2,0,0,0,2,0,0,0,0,0,41.809620,-72.830515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,WINSTED,7,2,3,1,0,1,0,2,5,1,0,1,1,41.921207,-73.060108
216,WOLCOTT,0,1,1,0,0,1,0,0,0,0,0,0,0,41.602320,-72.986772
217,WOODBRIDGE,3,0,1,1,0,0,0,2,0,0,1,0,0,41.352597,-73.008438
218,WOODBURY,2,0,0,0,0,2,0,1,0,0,0,0,0,41.544540,-73.209002


In [4]:
# Verify Counts
city_counts.count()

City                   220
Heroin                 220
Cocaine                220
Fentanyl               220
Oxycodone              220
Oxymorphone            220
Ethanol                220
Hydrocodone            220
Benzodiazepine         220
Methadone              220
Amphetamine            220
Tramadol               220
Morphine Not Heroin    220
Other                  220
Latitude               220
Longitude              220
dtype: int64

In [5]:
import gmaps
gmaps.configure(api_key=gkey)
# Want to center the map on CT, so we need coordinates for the "center" of CT.
ct_coord = (41.5032, -72.6877)

In [6]:
# Toggle Map functionality copied from Jupyter-Gmaps API documentation
# Altered to fit desired use
from IPython.display import display
import ipywidgets as widgets

class OutletExplorer(object):

  def __init__(self, df):
      """
      Jupyter widget for exploring drug deaths by drug

      Using checkboxes, the user chooses which drugs to include.
      """
      self._df = df
      self._heatmap = None


      title_widget = widgets.HTML(
          '<h3>Accidental Drug Deaths in Connecticut between 2012 and 2018</h3>'
          '<h4>Data from <a href="https://catalog.data.gov/dataset/accidental-drug-related-deaths-january-2012-sept-2015">Data.gov</a></h4>'
      )
      controls = self._render_controls(True, True, True, True, True, True, True)
      map_figure = self._render_map(True, True, True, True, True, True, True)
      self._container = widgets.VBox(
          [title_widget, controls, map_figure])

  def render(self):
      """ Render the widget """
      display(self._container)

  def _render_map(self, initial_include_heroin, initial_include_fentanyl, initial_include_cocaine, init_benzo, init_ethanol, init_oxycod, init_other):
      """ Render the initial map """
      fig = gmaps.figure(layout={'height': '500px'},center=ct_coord,zoom_level=8.4)
      weights = self._df[self._generate_weights(True, True, True, True, True, True, True)].sum(axis=1)
      self._heatmap_layer = gmaps.heatmap_layer(
          self._df[['Latitude','Longitude']],
          weights = weights,
          max_intensity = max(weights),
          point_radius = 40.0
          )
      fig.add_layer(self._heatmap_layer)
      return fig

  def _render_controls(
      self,
      initial_include_heroin,
      initial_include_fentanyl,
      initial_include_cocaine,
      init_benzo,
      init_ethanol,
      init_oxycod,
      init_other
  ):
      """ Render the checkboxes """
      self._heroin_checkbox = widgets.Checkbox(
          value=initial_include_heroin,
          description='Heroin'
      )
      self._fentanyl_checkbox = widgets.Checkbox(
          value=initial_include_fentanyl,
          description='Fentanyl'
      )
      self._cocaine_checkbox = widgets.Checkbox(
          value=initial_include_cocaine,
          description='Cocaine'
      )
      self._benzo_checkbox = widgets.Checkbox(
          value=init_benzo,
          description='Benzodiazepine'
      )
      self._ethanol_checkbox = widgets.Checkbox(
          value=init_ethanol,
          description='Ethanol'
      )
      self._oxycod_checkbox = widgets.Checkbox(
          value=init_oxycod,
          description='Oxycodone'
      )
      self._other_checkbox = widgets.Checkbox(
          value=init_other,
          description='Other'
      )
      self._heroin_checkbox.observe(
          self._on_controls_change, names='value')
      self._fentanyl_checkbox.observe(
          self._on_controls_change, names='value')
      self._cocaine_checkbox.observe(
          self._on_controls_change, names='value')
      self._benzo_checkbox.observe(
          self._on_controls_change, names='value')
      self._ethanol_checkbox.observe(
          self._on_controls_change, names='value')
      self._oxycod_checkbox.observe(
          self._on_controls_change, names='value')
      self._other_checkbox.observe(
          self._on_controls_change, names='value')
      controls = widgets.HBox(
          [self._heroin_checkbox, self._fentanyl_checkbox, self._cocaine_checkbox, self._benzo_checkbox, self._ethanol_checkbox, self._oxycod_checkbox, self._other_checkbox],
          layout=widgets.Layout(display='flex',justify_content='flex-start', flex_wrap='wrap'))
      return controls

  def _on_controls_change(self, obj):
      """
      Called when the checkboxes change

      This method builds the list of symbols to include on the map,
      based on the current checkbox values. It then updates the
      heatmap layer with the new cumulative weights.
      """
      include_heroin = self._heroin_checkbox.value
      include_fentanyl = self._fentanyl_checkbox.value
      include_cocaine = self._cocaine_checkbox.value
      include_benzo = self._benzo_checkbox.value
      include_ethanol = self._ethanol_checkbox.value
      include_oxycod = self._oxycod_checkbox.value
      include_other = self._other_checkbox.value
      drugs_list = self._generate_weights(
          include_heroin, include_fentanyl, include_cocaine, include_benzo, include_ethanol, include_oxycod, include_other)
      # Update the layer with the new weights:
      self._heatmap_layer.weights = self._df[drugs_list].sum(axis=1)

  def _generate_weights(self, include_heroin, include_fentanyl, include_cocaine, include_benzo, include_ethanol, include_oxycod, include_other):
      """ Generate the list of drugs to include """
      drugs_list = []
      if include_heroin:
          drugs_list.append('Heroin')
      if include_fentanyl:
          drugs_list.append('Fentanyl')
      if include_cocaine:
          drugs_list.append('Cocaine')
      if include_benzo:
          drugs_list.append('Benzodiazepine')
      if include_ethanol:
          drugs_list.append('Ethanol')
      if include_oxycod:
          drugs_list.append('Oxycodone')
      if include_other:
          drugs_list.append('Other')
      return drugs_list



OutletExplorer(city_counts).render()